## Library Import

In [1]:
import duckdb
import polars as p

import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import load_setting
settings = load_setting()

## Load Silver Data

In [2]:
# Test connection to Silver Data db
try:
    db_path = settings['process']['silver_db_path']
    print(f"Attempting to connect to: {db_path}")
    
    # Try connection
    con = duckdb.connect(f"../{db_path}")
    
    # Test Connection with a simple query
    result = con.execute("SELECT 1").fetchone()
    print("Connection Succesfful!")
    print(f"Test query result: {result}")
    
    
    # List all tables in the database
    tables = con.execute("SHOW TABLES").fetchall()
    print("\nAvailable tables:", tables)
    
except Exception as e:
    print(f"Connection Failed {str(e)}")

Attempting to connect to: process/storage/silver/resampled.duckdb
Connection Succesfful!
Test query result: (1,)

Available tables: [('gold_data',), ('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [3]:
# List all tables
tables = con.execute("SHOW TABLES").fetchall()
print("Available tables:", tables)

Available tables: [('gold_data',), ('raw_data',), ('silver_1',), ('silver_13',), ('silver_21',), ('silver_3',), ('silver_34',), ('silver_5',), ('silver_55',), ('silver_8',)]


In [4]:
# Get schema information for a specific table
# Replace 'your_table_name' with an actual table name
table_info = con.execute("DESCRIBE silver_1").fetchall()
print("Table schema:", table_info)

Table schema: [('symbol', 'VARCHAR', 'YES', None, None, None), ('date', 'TIMESTAMP WITH TIME ZONE', 'YES', None, None, None), ('open', 'DOUBLE', 'YES', None, None, None), ('high', 'DOUBLE', 'YES', None, None, None), ('low', 'DOUBLE', 'YES', None, None, None), ('close', 'DOUBLE', 'YES', None, None, None), ('volume', 'DOUBLE', 'YES', None, None, None)]


In [5]:
# Get sample data from the table
sample_data = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("Sample data:", sample_data)

Sample data: [('VRNS', datetime.datetime(2019, 1, 25, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 18.0833320618, 18.4699993134, 17.9133338928, 18.3733329773, 476700.0), ('VRNS', datetime.datetime(2019, 3, 5, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 19.6033325195, 20.2233333588, 19.4233322144, 20.0766677856, 1911600.0), ('VRNS', datetime.datetime(2019, 8, 8, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 22.8866672516, 23.703332901, 22.8866672516, 23.3566665649, 895200.0), ('VRNS', datetime.datetime(2019, 10, 29, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 23.4333324432, 24.6996669769, 22.3366661072, 23.8233337402, 4088700.0), ('VRNS', datetime.datetime(2019, 11, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 24.3366661072, 24.7833328247, 24.2466678619, 24.6700000763, 741900.0)]


In [6]:
# Get the date range of the silver_1 table
con.execute("SELECT MIN(date), MAX(date) FROM silver_1").fetchall() 

[(datetime.datetime(1962, 1, 2, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>),
  datetime.datetime(2025, 6, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>))]

In [7]:
# Get the first 5 rows of Resampled Data
first_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)

First 5 rows: [('YI', datetime.datetime(2025, 6, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 8.0, 8.3093, 8.0, 8.1794, 12970.0), ('JBDI', datetime.datetime(2025, 6, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 1.0, 1.02, 0.98, 1.02, 20529.0), ('NVGS', datetime.datetime(2025, 6, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 14.22, 14.48, 14.09, 14.37, 280670.0), ('DHX', datetime.datetime(2025, 6, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 2.54, 2.65, 2.49, 2.59, 99899.0), ('DJT', datetime.datetime(2025, 6, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 20.35, 20.43, 19.86, 19.93, 5550513.0)]


In [8]:
# Get total row count
row_count = con.execute("SELECT COUNT(*) FROM silver_1").fetchone()
print("Total rows:", row_count)

Total rows: (6184327,)


In [9]:
# Get the first 5 rows of the table
first_5_rows = con.execute("SELECT * FROM silver_1 LIMIT 5").fetchall()
print("First 5 rows:", first_5_rows)
# Get the last 5 rows of the table
last_5_rows = con.execute("SELECT * FROM silver_1 ORDER BY date DESC LIMIT 5").fetchall()
print("Last 5 rows:", last_5_rows)


First 5 rows: [('VRNS', datetime.datetime(2019, 1, 25, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 18.0833320618, 18.4699993134, 17.9133338928, 18.3733329773, 476700.0), ('VRNS', datetime.datetime(2019, 3, 5, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 19.6033325195, 20.2233333588, 19.4233322144, 20.0766677856, 1911600.0), ('VRNS', datetime.datetime(2019, 8, 8, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 22.8866672516, 23.703332901, 22.8866672516, 23.3566665649, 895200.0), ('VRNS', datetime.datetime(2019, 10, 29, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>), 23.4333324432, 24.6996669769, 22.3366661072, 23.8233337402, 4088700.0), ('VRNS', datetime.datetime(2019, 11, 12, 0, 0, tzinfo=<DstTzInfo 'America/Edmonton' MST-1 day, 17:00:00 STD>), 24.3366661072, 24.7833328247, 24.2466678619, 24.6700000763, 741900.0)]
Last 5 rows: [('YI', datetime.datetime(2025, 6, 12, 0, 0, tzinfo=<DstTzInfo 'Am

In [10]:
# Get column statistics
stats = con.execute("""
    SELECT 
        symbol,
        COUNT(*) as count,
        AVG(open) as avg,
        MIN(open) as min,
        MAX(open) as max
    FROM silver_1
    GROUP BY symbol
""").fetchall()
print("Column statistics:", stats)

Column statistics: [('DKL', 3171, 22.274408617188428, 3.4266770334, 49.7129155399), ('BKE', 8333, 9.469703608438854, 0.266297698, 50.5897320505), ('MDV', 836, 12.635892338615909, 7.3143748487, 50.0757185962), ('POAI', 3803, 21083.50224882896, 0.5550000072, 225000.0), ('ADTN', 7763, 15.022205683576434, 3.5078499372, 36.577340916), ('FONR', 10994, 20.096583987966532, 0.0, 114.84375), ('CIGL', 38, 5.511871062978948, 4.0, 7.68), ('SGMT', 480, 5.64912292391, 1.9099999666, 18.4200000763), ('FLNC', 909, 17.93362925965302, 3.5699999332, 38.1300010681), ('MCK', 7698, 119.58872296771351, 11.6705776292, 726.51), ('DIOD', 13190, 15.050174104725997, 0.0, 112.5999984741), ('PLXS', 9915, 31.48981249496273, 0.0, 170.3800048828), ('SATL', 1059, 4.371295550790463, 0.7080000043, 10.4899997711), ('VRTS', 4137, 115.3284747664408, 3.1210713464, 296.5187520242), ('ZKH', 373, 7.4009965117932985, 2.65, 20.8500003815), ('HFWA', 6818, 13.366643237856058, 3.4150906969, 30.838836899), ('DNTH', 1754, 10.89469212385

In [11]:
import polars as pl

# Turn stats into a pl dataframe with mixed types allowed
stats_df = pl.DataFrame(stats, 
                        schema={
                            "symbol": pl.Utf8,
                            "count": pl.Int64,
                            "avg": pl.Float64,
                            "min": pl.Float64,
                            "max": pl.Float64
                        })
print(stats_df)

shape: (1_458, 5)
┌────────┬───────┬──────────────┬──────────┬────────────┐
│ symbol ┆ count ┆ avg          ┆ min      ┆ max        │
│ ---    ┆ ---   ┆ ---          ┆ ---      ┆ ---        │
│ str    ┆ i64   ┆ f64          ┆ f64      ┆ f64        │
╞════════╪═══════╪══════════════╪══════════╪════════════╡
│ DKL    ┆ 3171  ┆ 22.274409    ┆ 3.426677 ┆ 49.712916  │
│ BKE    ┆ 8333  ┆ 9.469704     ┆ 0.266298 ┆ 50.589732  │
│ MDV    ┆ 836   ┆ 12.635892    ┆ 7.314375 ┆ 50.075719  │
│ POAI   ┆ 3803  ┆ 21083.502249 ┆ 0.555    ┆ 225000.0   │
│ ADTN   ┆ 7763  ┆ 15.022206    ┆ 3.50785  ┆ 36.577341  │
│ …      ┆ …     ┆ …            ┆ …        ┆ …          │
│ HAE    ┆ 8585  ┆ 35.362884    ┆ 5.9375   ┆ 140.220001 │
│ MCW    ┆ 996   ┆ 10.337259    ┆ 5.14     ┆ 23.67      │
│ KEN    ┆ 2618  ┆ 13.217301    ┆ 0.675694 ┆ 41.075122  │
│ BMGL   ┆ 76    ┆ 4.0065       ┆ 1.0      ┆ 8.315      │
│ BIYA   ┆ 58    ┆ 4.181552     ┆ 1.38     ┆ 7.05       │
└────────┴───────┴──────────────┴──────────┴──────────

/opt/homebrew/Cellar/python@3.10/3.10.17/Frameworks/Python.framework/Versions/3.10/lib/python3.10/functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


In [12]:
con.close()
